In [1]:
import tensorflow as tf 
from tensorflow import keras as keras
from tensorflow.python.keras import backend as K
import os
import numpy as np
import datetime
import importlib


import Clone_Model
import rsc

In [2]:
checkpoint_path = "./cloning/checkpoints/cloning/default_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

log_dir_clone = "./cloning/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_clone, histogram_freq=1)

In [39]:
#      CLONE MODEL EAGER ENABLED
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)

if not tf.executing_eagerly(): tf.compat.v1.enable_eager_execution()

K.clear_session()

batch_size = 3

train, test = rsc.get_titanic_dataset()


model = rsc.get_model_slim()

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    run_eagerly = tf.executing_eagerly()
    )


history = model.fit(train,
                    batch_size=batch_size,
                    epochs=1, 
                    steps_per_epoch=3, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])


clone = Clone_Model.satify_model(model)

clone.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())

sat_cb = Clone_Model.sat_results()

history = clone.fit(train,
                    batch_size=batch_size,
                    epochs = 1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback, sat_cb])
                

3/3 [==============================] - 0s 12ms/step - loss: 0.6941 - val_loss: 0.6961
Init Satlayer!
Init Satlayer!
Init Satlayer!
Init Satlayer!
Init Satlayer!
SatLayer.update(): Input tensor has shape: (None, 3)
SatLayer.update(): Input tensor has shape: (None, 3)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 3)
SatLayer.update(): Input tensor has shape: (None, 3)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 2)
Clone_Model.satify_model() end.
SatLayer.update(): Input tensor has shape: (20, 3)
SatLayer.update(): Input tensor has shape: (20, 3)
SatLayer.update(): Input tensor has shape: (20, 2)
SatLayer.update(): Input tensor has shape: (20, 2)
SatLayer.update(): Input tensor has shape: (20, 2)
1/3 [=========>.......

In [114]:
#      TRAIN MODEL EAGER ENABLED MNIST CLASSIFICATION
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)

if not tf.executing_eagerly(): tf.compat.v1.enable_eager_execution()

K.clear_session()

batch_size = 20

train, test = rsc.get_mnist()


model = rsc.get_model_unitlist(hidden_layers_spec=[128])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



history = model.fit(train[0],
                    train[1],
                    batch_size=batch_size,
                    epochs=10, 
                    steps_per_epoch=None, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])



Epoch 1/10
3000/3000 [==============================] - 12s 4ms/step - loss: 2.2026 - accuracy: 0.8638 - val_loss: 0.7533 - val_accuracy: 0.9000
Epoch 2/10
3000/3000 [==============================] - 10s 3ms/step - loss: 0.3796 - accuracy: 0.9107 - val_loss: 0.2871 - val_accuracy: 0.9000
Epoch 3/10
3000/3000 [==============================] - 9s 3ms/step - loss: 0.2980 - accuracy: 0.9265 - val_loss: 0.2013 - val_accuracy: 0.9000
Epoch 4/10
3000/3000 [==============================] - 9s 3ms/step - loss: 0.2702 - accuracy: 0.9352 - val_loss: 0.2948 - val_accuracy: 0.9500
Epoch 5/10
3000/3000 [==============================] - 8s 3ms/step - loss: 0.2509 - accuracy: 0.9410 - val_loss: 0.3036 - val_accuracy: 0.9000
Epoch 6/10
3000/3000 [==============================] - 7s 2ms/step - loss: 0.2457 - accuracy: 0.9428 - val_loss: 0.1963 - val_accuracy: 0.9000
Epoch 7/10
3000/3000 [==============================] - 10s 3ms/step - loss: 0.2393 - accuracy: 0.9457 - val_loss: 0.2068 - val_accura

In [175]:
#      CLONE MODEL EAGER ENABLED MNIST CLASSIFICATION
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)


clone = Clone_Model.satify_model(model)

"""clone.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              run_eagerly = tf.executing_eagerly())
"""
sat_cb = Clone_Model.sat_results()

history = clone.fit(train[0],
                    train[1],
                    batch_size=10,
                    epochs = 1,
                    steps_per_epoch=None,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback, sat_cb])
              

<class 'tensorflow.python.keras.layers.core.Flatten'>
RESET Weights!
RESET Weights!
Clone_Model.satify_model() end.
5997/6000 [============================>.] - ETA: 0s - loss: 1.8953 - accuracy: 0.9425 - observed_samples_sat_dense: 29988.4391 - observed_samples_sat_dense_1: 29989.5199
Trace of covmat: 319996.96106018237
Sum Evals comp: 319996.961060183
Sum Evals real part: 319996.961060183
CovMat Diagonal entries head: [-2.49498336e-06  9.95872674e-02  1.99976921e+04 -8.04238533e-07
  7.18065862e+02 -3.43009852e-07  3.24782042e+02  1.77163697e+04
  6.98610451e-07  3.83050517e-07  2.40536002e+04  7.92172094e+03
  4.77516521e+00 -1.58391955e-06  4.63762650e+03 -1.81244488e-06
 -7.15967021e-07  7.42302283e+03 -2.50923820e-06  1.77621058e+02]
Dim ratios: [0.09778792 0.19266287 0.28289361 0.36138586 0.43253538 0.49598976
 0.54054428 0.58208355 0.6194061  0.65543882 0.68967162 0.72249705
 0.7521237  0.77809526 0.80235387 0.82323711 0.84207499 0.85946089
 0.87524506 0.88961903 0.90376996 0.9

In [62]:
in_shape = model.layers[0].input_shape[-2:]
print(in_shape)
np.ones(shape=in_shape).shape

(28, 28)


(28, 28)

In [46]:
#(None, 21, 5)[1:]*2
#np.zeros(shape=(None))
importlib.reload(rsc)
train_b, test_b = rsc.get_titanic_dataset(batch_size=3)
print(train_b)
print(model(np.array([[1,2,3,4] ])))

for ix, x in enumerate(train_b):
    print(x)

<BatchDataset shapes: ((None, None, 4), (None, None, 2)), types: (tf.float32, tf.int32)>
tf.Tensor([[0.69211704 0.3078829 ]], shape=(1, 2), dtype=float32)
(<tf.Tensor: shape=(3, 20, 4), dtype=float32, numpy=
array([[[ 61.    ,   0.    ,   0.    ,  32.3208],
        [ 38.    ,   0.    ,   1.    , 153.4625],
        [ 36.    ,   0.    ,   0.    ,  26.3875],
        [ 28.    ,   0.    ,   0.    ,  26.55  ],
        [ 22.    ,   0.    ,   0.    ,  10.5167],
        [ 28.    ,   3.    ,   1.    ,  25.4667],
        [ 28.    ,   0.    ,   0.    ,   7.8958],
        [ 30.    ,   0.    ,   0.    ,   7.8958],
        [ 34.    ,   1.    ,   0.    ,  21.    ],
        [  3.    ,   4.    ,   2.    ,  31.3875],
        [ 31.    ,   1.    ,   0.    ,  57.    ],
        [ 16.    ,   0.    ,   0.    ,   8.05  ],
        [ 28.    ,   3.    ,   1.    ,  25.4667],
        [ 39.    ,   0.    ,   0.    ,   0.    ],
        [ 28.    ,   0.    ,   0.    , 227.525 ],
        [ 19.    ,   0.    ,   0.    ,   7

InvalidArgumentError: Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [7,4], [batch]: [20,4]